In [1]:
import tensorflow.compat.v1 as tf
import warnings
tf.disable_v2_behavior()
tf.set_random_seed(777)  # for reproducibility
warnings.filterwarnings('ignore')
print(tf.__version__)

Instructions for updating:
non-resource variables are not supported in the long term
2.4.0


In [2]:
import numpy as np
import random
import tensorflow as tf2 #Tensorflow 2.x

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# normalizing data
x_train, x_test = x_train / 255.0, x_test / 255.0

# change data shape
print(x_train.shape)  # (60000, 28, 28)

(60000, 28, 28)


In [4]:
learning_rate = 0.001
batch_size = 100
training_epochs = 15
nb_classes = 10
num_epochs = 50
num_iterations = 600

In [5]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])

y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

#print(x_train.shape)
#print(y_train.shape)
#print(x_test.shape)
#print(y_test.shape)

In [6]:
# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# weights & bias for nn layers
#W1 = tf.Variable(tf.random_normal([784, 256]))

keep_prob = tf.placeholder(tf.float32)

# Glorot normal initializer, also called Xavier normal initializer.
init = tf2.initializers.GlorotUniform()
W1 = tf.get_variable("W1", shape=[784, 512],
                     initializer=init)

b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[512, 512],
                     initializer=init)
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[512, 512],
                     initializer=init)
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("W4", shape=[512, 512],
                     initializer=init)
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("W5", shape=[512, 10],
                     initializer=init)
b5 = tf.Variable(tf.random_normal([10]))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=tf.stop_gradient(Y)))
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [8]:
correct_prediction = tf.equal(tf.argmax(hypothesis, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [9]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
for epoch in range(num_epochs):
    avg_cost = 0
    for iteration in range(num_iterations):
        batch_xs, batch_ys = next_batch(batch_size, x_train, y_train)
        _, cost_val = sess.run([train, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
        avg_cost += cost_val / num_iterations
    print(f"Epoch: {(epoch + 1):04d}, Cost: {avg_cost:.9f}")
print("Learning Finished!")
# Test model and check accuracy
print(
    "Accuracy:",
    sess.run(accuracy, feed_dict={X: x_test, Y: y_test,keep_prob: 1}),
)

Epoch: 0001, Cost: 0.434862185
Epoch: 0002, Cost: 0.160535965
Epoch: 0003, Cost: 0.118981319
Epoch: 0004, Cost: 0.099807105
Epoch: 0005, Cost: 0.087279697
Epoch: 0006, Cost: 0.077537858
Epoch: 0007, Cost: 0.071789958
Epoch: 0008, Cost: 0.063589609
Epoch: 0009, Cost: 0.061542962
Epoch: 0010, Cost: 0.053375779
Epoch: 0011, Cost: 0.055632014
Epoch: 0012, Cost: 0.049608099
Epoch: 0013, Cost: 0.047286633
Epoch: 0014, Cost: 0.045896992
Epoch: 0015, Cost: 0.044382787
Epoch: 0016, Cost: 0.042951799
Epoch: 0017, Cost: 0.039908521
Epoch: 0018, Cost: 0.041425163
Epoch: 0019, Cost: 0.037230770
Epoch: 0020, Cost: 0.038737402
Epoch: 0021, Cost: 0.037593434
Epoch: 0022, Cost: 0.034973617
Epoch: 0023, Cost: 0.034986246
Epoch: 0024, Cost: 0.034306969
Epoch: 0025, Cost: 0.033164704
Epoch: 0026, Cost: 0.033341510
Epoch: 0027, Cost: 0.032722052
Epoch: 0028, Cost: 0.031390321
Epoch: 0029, Cost: 0.031576875
Epoch: 0030, Cost: 0.029973103
Epoch: 0031, Cost: 0.031672061
Epoch: 0032, Cost: 0.032885177
Epoch: 0

In [12]:
# Predict
correct_num = 0

for x in range(0, 10):
    r = random.randint(0, x_test.shape[0]-1)
    print("#",x)
    real=sess.run(tf.argmax(y_test[r]))
    predict=sess.run(tf.argmax(hypothesis, axis=1), feed_dict={X: x_test[r : r + 1], keep_prob: 1})
    print("Label: ",real)
    print("Prediction: ",predict)

    if real == predict:
        correct_num+=1
    print("Accuracy: ", correct_num/(x+1))

print("Total Accuracy: ", correct_num/(x+1))


# 0
Label:  1
Prediction:  [1]
Accuracy:  1.0
# 1
Label:  8
Prediction:  [8]
Accuracy:  1.0
# 2
Label:  8
Prediction:  [8]
Accuracy:  1.0
# 3
Label:  8
Prediction:  [8]
Accuracy:  1.0
# 4
Label:  6
Prediction:  [6]
Accuracy:  1.0
# 5
Label:  0
Prediction:  [0]
Accuracy:  1.0
# 6
Label:  7
Prediction:  [7]
Accuracy:  1.0
# 7
Label:  0
Prediction:  [0]
Accuracy:  1.0
# 8
Label:  7
Prediction:  [7]
Accuracy:  1.0
# 9
Label:  8
Prediction:  [8]
Accuracy:  1.0
Total Accuracy:  1.0
